## Libraries

In [3]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy import signal
from scipy import stats
import numpy as np
from pylab import specgram
from scipy.io import wavfile
from scipy.signal import hilbert, chirp
import pandas as pd

from skimage import util

import librosa
from librosa import feature
import re
import plotly.offline as offline
import plotly.graph_objs as go
from PIL import Image
import time
import math

## Data loading

In [4]:
path = "" #path to data downloaded on mini vast challenge website
os.chdir(path)
birds=pd.read_csv('AllBirdsv4.csv')
test_birds=pd.read_csv('Test Birds Location.csv')

## Extract sound features for the train set

In [ ]:
English_name = []
file_id = []

skewness = []
kurto = []
rmse = []
mfcc = []

chroma_stft = []
contrast = []
tonnetz = []
melspec = []

for filename in os.listdir(r"birdswav"): 
    match = re.search(r'(\w+-){1,}(?=[0-9])', filename)
    English_name.append(match.group()[:-1])
    file_id.append(int(filename[match.span()[1]:-4]))
    
    y, sr = librosa.load(r'birdswav/%s'%filename, sr = None)
    y = y[:sr]
    
    M = math.ceil(sr * 1024 / 48000) #20 ms environ (cf methodo du papier)
    
    if np.ndim(y)>1:
        y = np.mean(y, axis=1)
    
    while y.shape[0] < sr:
        y = np.append(y,y)
    
    if y.shape[0]>=sr:
        y.resize((sr,))
    
    kurto.append(stats.kurtosis(y, axis = 0))
    skewness.append(stats.skew(y, axis = 0))
    
    rmse.append(feature.rmse(y=y, frame_length = M))
    
    mfcc.append(feature.mfcc(y=y, sr=sr,n_mfcc = 30))

    chroma_stft.append(feature.chroma_stft(y=y, sr=sr))

    contrast.append(feature.spectral_contrast(y=y, sr=sr, n_fft = M))

    tonnetz.append(feature.tonnetz(y=y, sr=sr))
    
    melspec.append(feature.melspectrogram(y=y, sr=sr, n_fft = M))

C:\Users\ambro\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning:

Trying to estimate tuning from empty frequency set.

C:\Users\ambro\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning:

Trying to estimate tuning from empty frequency set.



In [ ]:
data = [('name', English_name),
         ('id', file_id),
         ('kurto', kurto),
         ('skewness', skewness),
         ('mfcc', mfcc),
         ('rmse', rmse),
         ('chroma_stft', chroma_stft),
         ('contrast', contrast),
         ('tonnetz', tonnetz),
         ('melspec', melspec),
         ]

In [ ]:
df = pd.DataFrame.from_items(data)

In [ ]:
for k in df.columns:
    if np.ndim(df.loc[0,k]) == 0:
        continue
    
    long_k = []
    for i in range(2081):
        long_k.append(df.loc[i,str(k)].shape[1])
    
    for j in range(df.loc[0,k].shape[0]):
        for l in range(np.min(long_k)):
            df[str(k) + str(j) + "_" + str(l)] = list(df.loc[i,k][j,l] for i in range(df.shape[0]))
    df = df.drop(columns = k)

# drop features that are objects

stft_ = []
for i in df.columns : 
    if 'stft' in i :
        stft_.append(i)

df = df.drop( stft_ , axis =1 )
df = df.drop( ['melspec'], axis =1 )

In [60]:
df.to_csv('features_v2.csv')

## Feature extraction (Kasios)

In [3]:
number = []

skewness = []
kurto = []
rmse = []
mfcc = []

chroma_stft = []
contrast = []
tonnetz = []
melspec = []

for filename in os.listdir(r"kasios"):
    match = re.search(r'([0-9]+)', filename)
    number.append(match.group())
    
    y, sr = librosa.load(r'kasios/%s'%filename, sr = None)
    y = y[:sr]
    
    M = math.ceil(sr * 1024 / 48000) #20 ms environ (cf methodo du papier)
    
    if np.ndim(y)>1:
        y = np.mean(y, axis=1)
    
    while y.shape[0] < sr:
        y = np.append(y,y)
    
    if y.shape[0]>=sr:
        y.resize((sr,))
    
    kurto.append(stats.kurtosis(y, axis = 0))
    skewness.append(stats.skew(y, axis = 0))
    
    rmse.append(feature.rmse(y=y, frame_length = M))
    
    mfcc.append(feature.mfcc(y=y, sr=sr,n_mfcc = 30))

    chroma_stft.append(feature.chroma_stft(y=y, sr=sr))

    contrast.append(feature.spectral_contrast(y=y, sr=sr, n_fft = M))

    tonnetz.append(feature.tonnetz(y=y, sr=sr))
    
    melspec.append(feature.melspectrogram(y=y, sr=sr, n_fft = M))

C:\Users\ambro\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning:

Trying to estimate tuning from empty frequency set.



In [5]:
data2 = [('numero', number),
         ('kurto', kurto),
         ('skewness', skewness),
         ('mfcc', mfcc),
         ('rmse', rmse),
         ('chroma_stft', chroma_stft),
         ('contrast', contrast),
         ('tonnetz', tonnetz),
         ('melspec', melspec),
         ]

df_kasios = pd.DataFrame.from_items(data2)


C:\Users\ambro\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.



In [50]:
df_kasios.to_csv("features_v2_kasios.csv")

In [9]:
for k in df_kasios.columns:
    if np.ndim(df_kasios.loc[0,k]) == 0:
        continue
    
    long_k = []
    for i in range(15):
        long_k.append(df_kasios.loc[i,str(k)].shape[1])
    
    for j in range(df_kasios.loc[0,k].shape[0]):
        for l in range(np.min(long_k)):
            df_kasios[str(k) + str(j) + "_" + str(l)] = list(df_kasios.loc[i,k][j,l] for i in range(df_kasios.shape[0]))
    df_kasios_fin = df_kasios.drop(columns = k)

In [13]:
df_kasios_fin.to_csv("features_v2_kasios.csv")